In [11]:
import os
from pathlib import Path
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories
from src.project import logger
import joblib
from dataclasses import dataclass
from sklearn.linear_model import ElasticNet
import pandas as pd

In [2]:
%pwd

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\13-End-to-end-DS-Project\\resesarch'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\13-End-to-end-DS-Project'

In [7]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [29]:
class ConfigurationManager: 
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config


In [33]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [32]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-06-22 20:29:35,886: INFO: common: yaml file: src\project\config\config.yaml loaded successfully]
[2025-06-22 20:29:35,889: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-22 20:29:35,893: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-22 20:29:35,895: INFO: common: Created Directory at: artifacts]
[2025-06-22 20:29:35,897: INFO: common: Created Directory at: artifacts/model_trainer]
quality
